# Evaluate Stitched IP

Evaluate the Stitched_IP with RTL simulation. It takes a lot of time to test 1 example, so use it with very few representative examples.

If follows this [FINN repo](https://github.com/Xilinx/finn/blob/main/src/finn/builder/build_dataflow_steps.py#L220) function:
```python
def prepare_for_stitched_ip_rtlsim(verify_model, cfg)
```

In [ ]:
import os

import numpy as np
from qonnx.core.modelwrapper import ModelWrapper

import finn.core.onnx_exec as oxe

from qonnx.custom_op.registry import getCustomOp

from finn.util.basic import pynq_part_map
import pandas as pd

from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP

from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim

from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.analysis.fpgadataflow.dataflow_performance import dataflow_performance
from finn.util.basic import (
    get_rtlsim_trace_depth,
    pyverilate_get_liveness_threshold_cycles,
)

# Setup Folders and Model Filenames

In [ ]:
experiment_folder = ('./experiments'
                     + '/750_FPS'
                     + '/01_full_build'
                     + '/output_full_build')
model_folder = experiment_folder + '/intermediate_models'

qonnx_to_finn = model_folder + '/custom_step_add_pre_proc.onnx'

stitched_ip = model_folder + '/step_create_stitched_ip.onnx'
parent_model = model_folder + '/dataflow_parent.onnx'

stitched_rtlsim_filename = "./eval_stitched_ip/stitched_rtlsim.onnx"

# Function to prepare model for RTLSim

FIFOs impl_style from "vivado" to "rtl"

In [ ]:
def prepare_for_stitched_ip_rtlsim(verify_model, fpga_part, target_clk_ns):
    need_restitch = False
    # switch impl_style=vivado components to rtl
    # StreamingFIFO must have impl_style=rtl
    for fifo_layer in verify_model.get_nodes_by_op_type("StreamingFIFO_rtl"):
        inst = getCustomOp(fifo_layer)
        if inst.get_nodeattr("impl_style") != "rtl":
            inst.set_nodeattr("impl_style", "rtl")
            inst.set_nodeattr("code_gen_dir_ipgen", "")
            inst.set_nodeattr("ipgen_path", "")
            print(f'{fifo_layer.name} changed to RTL')
            need_restitch = True
    # if we've made alterations to the model, need to do some re-prep
    if need_restitch:
        print("Need to regen/re-stitch some IP for STITCHED_IP_RTLSIM")
        verify_model = verify_model.transform(
            PrepareIP(fpga_part, target_clk_ns)
        )
        verify_model = verify_model.transform(HLSSynthIP())
        verify_model = verify_model.transform(
            CreateStitchedIP(
                fpga_part,
                target_clk_ns,
                vitis=False,
            )
        )
  
    # set top-level prop for stitched-ip rtlsim and launch
    verify_model.set_metadata_prop("exec_mode", "rtlsim")
    # TODO make configurable
    # verify_model.set_metadata_prop("rtlsim_trace", "trace.vcd")
    return verify_model

# PYNQ Boards Available

In [ ]:
fpga_df = pd.DataFrame(pynq_part_map.items(), columns=['Board', 'FPGA Part'])
fpga_df

In [ ]:
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
print(fpga_part)

In [ ]:
synth_clk_period_ns = 10.0

# Create New Stitched IP with impl_style = "rtl"

It takes a long time to finish. Only needed the first time, to create the new stitched IP. Once it is created, this step can be skipped.

In [ ]:
# child_model = ModelWrapper(stitched_ip)

# child_model = prepare_for_stitched_ip_rtlsim(child_model, fpga_part, synth_clk_period_ns)

# child_model = child_model.transform(PrepareRTLSim())
# child_model.save(stitched_rtlsim_filename)

# Load Parent Model and link Partition (Child) to Stitched IP for RTLSim

In [ ]:
# parent model
model_for_rtlsim = ModelWrapper(parent_model)
# reference child model
sdp_node = getCustomOp(model_for_rtlsim.graph.node[1])
sdp_node.set_nodeattr("model", stitched_rtlsim_filename)

# LIVENESS_THRESHOLD 

Adjust it to model Max Cycles -> Critical Path.

In [ ]:
stiched_rtlsim_model = ModelWrapper(stitched_rtlsim_filename)
stiched_rtlsim_model = stiched_rtlsim_model.transform(AnnotateCycles())
estimate_network_performance = stiched_rtlsim_model.analysis(dataflow_performance)
prev_liveness = pyverilate_get_liveness_threshold_cycles()

In [ ]:
print(f'Previous LIVENESS_THRESHOLD = {prev_liveness}')
print(f'Estimated performance of Stuched_RTLSim\n{estimate_network_performance}')

In [ ]:
os.environ["LIVENESS_THRESHOLD"] = str(
    int(estimate_network_performance["critical_path_cycles"])
)

# Change it to previuous value
#os.environ["LIVENESS_THRESHOLD"] = str(prev_liveness)

In [ ]:
print(os.environ["LIVENESS_THRESHOLD"])

# Eval with DFire MINI

### MINI Dataset

In [ ]:
DS_FOLDER_MINI = '../../datasets/dfire_mini/'
#### Needed for DatasetMINI
DS_FOLDER_MINI += 'test/'

DS_IMGS_MINI = DS_FOLDER_MINI + 'images/'
DS_LABELS_MINI = DS_FOLDER_MINI + 'labels/'

In [ ]:
MINI_IMGS_LIST = sorted([DS_IMGS_MINI + img_file for img_file in os.listdir(DS_IMGS_MINI)])

In [ ]:
print(f'Number of test samples: {len(MINI_IMGS_LIST)}')

In [ ]:
import my_metrics as metrics

### Eval QONNX TO FINN model

In [ ]:
qonnx_to_finn_model = ModelWrapper(qonnx_to_finn)

### MINI

In [ ]:
qonnx_to_finn_metrics_MINI = metrics.eval_quant_onnx(
    imgs_list = MINI_IMGS_LIST,
    labels_dir = DS_LABELS_MINI,
    model_wrapped = qonnx_to_finn_model,
    bipolar=True,
    divide_255=False) # Preprocess included

In [ ]:
qonnx_to_finn_MINI_df = pd.DataFrame(qonnx_to_finn_metrics_MINI)
qonnx_to_finn_MINI_df

### Eval Stitched IP model

In [ ]:
stitched_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = model_for_rtlsim,
    bipolar=True,
    divide_255=False) # Preprocess included in FINN model

# Print Results

In [ ]:
qonnx_to_finn_df = pd.DataFrame(qonnx_to_finn_metrics)
qonnx_to_finn_df

In [ ]:
stitched_df = pd.DataFrame(stitched_metrics)
stitched_df